In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ✅ 유지 vs 해지 고객 지도 시각화 대시보드 (전문가 기법 + Folium + Geo + 필터 UI)

import geopandas as gpd
import pandas as pd
import folium
import os
from shapely.geometry import Point
from folium.plugins import MarkerCluster
from IPython.display import display, IFrame
from ipywidgets import Dropdown, Button, VBox, Output
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe

# ✅ Google Sheets 인증 및 불러오기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 유지 고객
sheet_url_유지 = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit"
df_u = get_as_dataframe(gc.open_by_url(sheet_url_유지).worksheet("3월마감 조건추출 위경도기초 선택컬럼_2403"), evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_u['상태'] = '3월마감 조건추출 위경도기초 선택컬럼_2403'

# ✅ 해지 고객
sheet_url_해지 = "https://docs.google.com/spreadsheets/d/1HVKhkyki9h3bOdI9RPhZsvQh3pjEcexdl1ISZiXX1lk/edit"
df_h = get_as_dataframe(gc.open_by_url(sheet_url_해지).worksheet("해지DB250101-0415"), evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_h['상태'] = '해지DB250101-0415'

# ✅ 공통 컬럼 정제 후 병합
공통컬럼 = ['관리지사명', '계약번호', '설치주소', '위도', '경도', '시군구', '읍면동', '상태']
df_u = df_u[[c for c in 공통컬럼 if c in df_u.columns]]
df_h = df_h[[c for c in 공통컬럼 if c in df_h.columns]]

# ✅ 위경도 정제 및 병합
for df_temp in [df_u, df_h]:
    df_temp['위도'] = pd.to_numeric(df_temp['위도'], errors='coerce')
    df_temp['경도'] = pd.to_numeric(df_temp['경도'], errors='coerce')

df = pd.concat([df_u, df_h]).dropna(subset=['위도', '경도'])

# ✅ SHP 파일 로드
sido_shp_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
gdf_sido = gpd.read_file(sido_shp_path, encoding='cp949')
gdf_sido = gdf_sido.set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

emd_shp_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"
gdf_emd = gpd.read_file(emd_shp_path, encoding='cp949')
gdf_emd = gdf_emd.set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

# ✅ 관리지사 드롭다운
지사목록 = sorted(df['관리지사명'].dropna().unique().tolist())
지사_선택 = Dropdown(options=['전체'] + 지사목록, description='지사선택:')
실행버튼 = Button(description="지도 보기", button_style='info')
output = Output()

# ✅ 실행 함수 (색상: 유지=파랑, 해지=빨강)
def 시각화_지도(b):
    with output:
        output.clear_output()

        if 지사_선택.value != '전체':
            필터 = df[df['관리지사명'] == 지사_선택.value].copy()
        else:
            필터 = df.copy()

        if 필터.empty:
            print("❌ 조건에 맞는 고객이 없습니다.")
            return

        m = folium.Map(location=[필터['위도'].mean(), 필터['경도'].mean()], zoom_start=11)

        # 행정경계
        folium.GeoJson(
            gdf_sido, tooltip=folium.GeoJsonTooltip(fields=['CTP_KOR_NM'], aliases=['시도'])
        ).add_to(m)

        folium.GeoJson(
            gdf_emd, tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases=['읍면동'])
        ).add_to(m)

        cluster = MarkerCluster().add_to(m)
        for _, row in 필터.iterrows():
            색상 = 'blue' if row['상태'] == '유지' else 'red'
            popup_html = f"""
                <b>상태:</b> {row['상태']}<br>
                <b>지사:</b> {row['관리지사명']}<br>
                <b>계약번호:</b> {row['계약번호']}<br>
                <b>설치주소:</b> {row['설치주소']}<br>
                <b>시군구:</b> {row.get('시군구', '')}<br>
                <b>읍면동:</b> {row.get('읍면동', '')}<br>
            """
            folium.CircleMarker(
                location=[row['위도'], row['경도']],
                radius=5,
                color=색상,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(cluster)

        m.save("유지_해지_고객지도.html")
        display(IFrame("유지_해지_고객지도.html", width=950, height=600))

# ✅ 버튼 바인딩 및 UI 표시
실행버튼.on_click(시각화_지도)
display(VBox([지사_선택, 실행버튼, output]))


<ipython-input-5-82cb38b21884>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['위도'] = pd.to_numeric(df_temp['위도'], errors='coerce')
<ipython-input-5-82cb38b21884>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['경도'] = pd.to_numeric(df_temp['경도'], errors='coerce')


In [ ]:
print(필터[['관리지사명', '위도', '경도']].head())
print(필터.shape)

NameError: name '필터' is not defined

In [ ]:
# ✅ 유지 vs 해지 고객 지도 시각화 대시보드 (전문가 기법 + Folium + Geo + 필터 UI)

import geopandas as gpd
import pandas as pd
import folium
import os
from shapely.geometry import Point
from folium.plugins import MarkerCluster
from IPython.display import display, IFrame
from ipywidgets import Dropdown, Button, VBox, Output
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe

# ✅ Google Sheets 인증 및 데이터 불러오기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 유지 고객 데이터
sheet_url_유지 = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit"
df_u = get_as_dataframe(gc.open_by_url(sheet_url_유지).worksheet("3월마감 조건추출 위경도기초 선택컬럼_2403"), evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_u['상태'] = '유지'

# ✅ 해지 고객 데이터
sheet_url_해지 = "https://docs.google.com/spreadsheets/d/1HVKhkyki9h3bOdI9RPhZsvQh3pjEcexdl1ISZiXX1lk/edit"
df_h = get_as_dataframe(gc.open_by_url(sheet_url_해지).worksheet("해지DB250101-0415"), evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_h['상태'] = '해지'

# ✅ 공통 컬럼 추출 및 병합
공통컬럼 = ['관리지사명', '계약번호', '설치주소', '위도', '경도', '시군구', '읍면동', '상태']
df_u = df_u[[c for c in 공통컬럼 if c in df_u.columns]]
df_h = df_h[[c for c in 공통컬럼 if c in df_h.columns]]

for df_temp in [df_u, df_h]:
    df_temp['위도'] = pd.to_numeric(df_temp['위도'], errors='coerce')
    df_temp['경도'] = pd.to_numeric(df_temp['경도'], errors='coerce')

# ✅ 데이터 병합
df = pd.concat([df_u, df_h]).dropna(subset=['위도', '경도'])

# ✅ SHP 파일 로딩 및 좌표계 설정
sido_shp_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
gdf_sido = gpd.read_file(sido_shp_path, encoding='cp949')
gdf_sido = gdf_sido.set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

emd_shp_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"
gdf_emd = gpd.read_file(emd_shp_path, encoding='cp949')
gdf_emd = gdf_emd.set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

# ✅ 드롭다운 UI
지사목록 = sorted(df['관리지사명'].dropna().unique().tolist())
지사_선택 = Dropdown(options=['전체'] + 지사목록, description='지사선택:')
실행버튼 = Button(description="지도 보기", button_style='info')
output = Output()

# ✅ 시각화 함수
def 시각화_지도(b):
    with output:
        output.clear_output()

        if 지사_선택.value != '전체':
            필터 = df[df['관리지사명'] == 지사_선택.value].copy()
        else:
            필터 = df.copy()

        if 필터.empty:
            print("❌ 조건에 맞는 고객이 없습니다.")
            return

        m = folium.Map(location=[필터['위도'].mean(), 필터['경도'].mean()], zoom_start=11)

        folium.GeoJson(
            gdf_sido, tooltip=folium.GeoJsonTooltip(fields=['CTP_KOR_NM'], aliases=['시도'])
        ).add_to(m)

        folium.GeoJson(
            gdf_emd, tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases=['읍면동'])
        ).add_to(m)

        상태_색상 = {'유지': 'blue', '해지': 'red'}
        cluster = MarkerCluster().add_to(m)
        for _, row in 필터.iterrows():
            색상 = 상태_색상.get(row['상태'], 'gray')
            popup_html = f"""
                <b>상태:</b> {row['상태']}<br>
                <b>지사:</b> {row['관리지사명']}<br>
                <b>계약번호:</b> {row['계약번호']}<br>
                <b>설치주소:</b> {row['설치주소']}<br>
                <b>시군구:</b> {row.get('시군구', '')}<br>
                <b>읍면동:</b> {row.get('읍면동', '')}<br>
            """
            folium.CircleMarker(
                location=[row['위도'], row['경도']],
                radius=5,
                color=색상,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(cluster)

        m.save("유지_해지_고객지도.html")
        display(IFrame("유지_해지_고객지도.html", width=950, height=600))

# ✅ UI 렌더링
실행버튼.on_click(시각화_지도)
display(VBox([지사_선택, 실행버튼, output]))


<ipython-input-7-2a01f18619d3>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['위도'] = pd.to_numeric(df_temp['위도'], errors='coerce')
<ipython-input-7-2a01f18619d3>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['경도'] = pd.to_numeric(df_temp['경도'], errors='coerce')


In [ ]:
from google.colab import files
files.download("유지_해지_고객지도.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ✅ 마커 수 제한 (예: 500개까지만 시각화)
필터_샘플 = 필터.sample(n=min(500, len(필터)), random_state=42)

for _, row in 필터_샘플.iterrows():
    색상 = 'blue' if row['상태'] == '유지' else 'red'
    ...

In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# ✅ 인증 정보
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 유지 고객 스프레드시트 → CSV 저장
url_유지 = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit"
sheet_유지 = gc.open_by_url(url_유지).worksheet("3월마감 조건추출 위경도기초 선택컬럼_2403")
df_유지 = get_as_dataframe(sheet_유지, evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_유지.to_csv("/content/유지고객_원본.csv", index=False, encoding='utf-8-sig')

# ✅ 해지 고객 스프레드시트 → CSV 저장
url_해지 = "https://docs.google.com/spreadsheets/d/1HVKhkyki9h3bOdI9RPhZsvQh3pjEcexdl1ISZiXX1lk/edit"
sheet_해지 = gc.open_by_url(url_해지).worksheet("해지DB250101-0415")
df_해지 = get_as_dataframe(sheet_해지, evaluate_formulas=True, dtype=str).dropna(how='all', axis=1)
df_해지.to_csv("/content/해지고객_원본.csv", index=False, encoding='utf-8-sig')

print("✅ CSV 파일로 저장 완료!")

✅ CSV 파일로 저장 완료!


In [ ]:
import os

# 찾을 경로들
paths_to_check = ["/content", "/mnt/data"]

# CSV 파일 검색
csv_files = []
for path in paths_to_check:
    if os.path.exists(path):
        csv_files += [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".csv")]

# 출력
if csv_files:
    print("✅ CSV 파일 목록:")
    for f in csv_files:
        print("-", f)
else:
    print("❌ CSV 파일이 없습니다.")

❌ CSV 파일이 없습니다.


In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

# ✅ CSV 파일 불러오기
df_u = pd.read_csv("/content/유지고객_원본.csv")
df_h = pd.read_csv("/content/해지고객_원본.csv")

# ✅ 공통 컬럼 + 상태 부여
df_u["상태"] = "유지"
df_h["상태"] = "해지"

공통컬럼 = ['관리지사명', '계약번호', '설치주소', '위도', '경도', '시군구', '읍면동', '상태']
df = pd.concat([df_u[공통컬럼], df_h[공통컬럼]], ignore_index=True)

# ✅ 위경도 숫자 변환 및 결측 제거
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 시도 및 읍면동 SHP 파일 불러오기
sido_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
emd_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"

gdf_sido = gpd.read_file(sido_path, encoding='cp949').set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")
gdf_emd = gpd.read_file(emd_path, encoding='cp949').set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

# ✅ 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=7)

# ✅ 행정경계 추가
folium.GeoJson(gdf_sido, tooltip=folium.GeoJsonTooltip(fields=['CTP_KOR_NM'], aliases=['시도'])).add_to(m)
folium.GeoJson(gdf_emd, tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases=['읍면동'])).add_to(m)

# ✅ 고객 마커 클러스터 추가
cluster = MarkerCluster().add_to(m)

for _, row in df.iterrows():
    color = 'blue' if row['상태'] == '유지' else 'red'
    popup_html = f"""
        <b>상태:</b> {row['상태']}<br>
        <b>지사:</b> {row['관리지사명']}<br>
        <b>계약번호:</b> {row['계약번호']}<br>
        <b>주소:</b> {row['설치주소']}<br>
        <b>시군구:</b> {row.get('시군구', '')}<br>
        <b>읍면동:</b> {row.get('읍면동', '')}
    """
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

# ✅ 저장 및 미리보기
m.save("유지_해지_고객지도_시각화.html")
m

FileNotFoundError: [Errno 2] No such file or directory: '/content/유지고객_원본.csv'

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

# ✅ CSV 파일 불러오기
df_u = pd.read_csv("/content/유지고객_원본.csv", dtype=str)
df_h = pd.read_csv("/content/해지고객_원본.csv", dtype=str)

df_u["상태"] = "유지"
df_h["상태"] = "해지"

# ✅ 필요한 컬럼 보정: 없는 컬럼은 빈 문자열로 생성
공통컬럼 = ['관리지사명', '계약번호', '설치주소', '위도', '경도', '시군구', '읍면동', '상태']
for col in 공통컬럼:
    if col not in df_u.columns:
        df_u[col] = ''
    if col not in df_h.columns:
        df_h[col] = ''

# ✅ 컬럼 맞춰서 병합
df = pd.concat([df_u[공통컬럼], df_h[공통컬럼]], ignore_index=True)

# ✅ 위경도 숫자 변환 및 결측 제거
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 시도 및 읍면동 SHP 파일 불러오기
sido_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
emd_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"

gdf_sido = gpd.read_file(sido_path, encoding='cp949').set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")
gdf_emd = gpd.read_file(emd_path, encoding='cp949').set_crs("EPSG:5179", allow_override=True).to_crs("EPSG:4326")

# ✅ 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=7)

# ✅ 행정경계 추가
folium.GeoJson(gdf_sido, tooltip=folium.GeoJsonTooltip(fields=['CTP_KOR_NM'], aliases=['시도'])).add_to(m)
folium.GeoJson(gdf_emd, tooltip=folium.GeoJsonTooltip(fields=['EMD_KOR_NM'], aliases=['읍면동'])).add_to(m)

# ✅ 고객 마커 클러스터
cluster = MarkerCluster().add_to(m)

for _, row in df.iterrows():
    color = 'blue' if row['상태'] == '유지' else 'red'
    popup_html = f"""
        <b>상태:</b> {row['상태']}<br>
        <b>지사:</b> {row['관리지사명']}<br>
        <b>계약번호:</b> {row['계약번호']}<br>
        <b>주소:</b> {row['설치주소']}<br>
        <b>시군구:</b> {row.get('시군구', '')}<br>
        <b>읍면동:</b> {row.get('읍면동', '')}
    """
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

# ✅ 저장 및 미리보기
m.save("유지해지_고객지도_최종.html")
m

In [ ]:
import os

경로 = "/content"
for file in os.listdir(경로):
    if "고객" in file:
        print("✅ 파일 발견:", file)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe

# ✅ 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Key/credentials.json", scope)
gc = gspread.authorize(credentials)

# ✅ 유지고객 스프레드시트 URL 및 시트명
유지_url = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit"
유지_시트 = "3월마감 조건추출 위경도기초 선택컬럼_2403"

# ✅ 불러오기 및 저장
ws_u = gc.open_by_url(유지_url).worksheet(유지_시트)
df_u = get_as_dataframe(ws_u, dtype=str)
df_u.to_csv("/content/drive/MyDrive/유지고객_원본.csv", index=False)
print("✅ 유지고객 CSV 저장 완료: /content/drive/MyDrive/유지고객_원본.csv")

✅ 유지고객 CSV 저장 완료: /content/drive/MyDrive/유지고객_원본.csv


In [ ]:
# ✅ 해지고객 스프레드시트 URL 및 시트명
해지_url = "https://docs.google.com/spreadsheets/d/1HVKhkyki9h3bOdI9RPhZsvQh3pjEcexdl1ISZiXX1lk/edit"
해지_시트 = "해지DB250101-0415"

# ✅ 불러오기 및 저장
ws_h = gc.open_by_url(해지_url).worksheet(해지_시트)
df_h = get_as_dataframe(ws_h, dtype=str)
df_h.to_csv("/content/drive/MyDrive/해지고객_원본.csv", index=False)
print("✅ 해지고객 CSV 저장 완료: /content/drive/MyDrive/해지고객_원본.csv")

✅ 해지고객 CSV 저장 완료: /content/drive/MyDrive/해지고객_원본.csv


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
from IPython.display import display, IFrame
from ipywidgets import Dropdown, Button, VBox, Output

# ✅ 1. CSV 불러오기
df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")
df_u['상태'] = '유지'

df_h = pd.read_csv("/content/drive/MyDrive/지도시각화/해지고객_원본.csv")
df_h['상태'] = '해지'

# ✅ 2. 병합 + 좌표 숫자 변환
df = pd.concat([df_u, df_h], ignore_index=True)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 3. SHP 파일 불러오기 및 좌표계 변환
sido_shp_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
gdf_sido = gpd.read_file(sido_shp_path, encoding='cp949')
gdf_sido = gdf_sido.set_crs("EPSG:5179").to_crs("EPSG:4326")

emd_shp_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"
gdf_emd = gpd.read_file(emd_shp_path, encoding='cp949')
gdf_emd = gdf_emd.set_crs("EPSG:5179").to_crs("EPSG:4326")

# ✅ 4. UI 구성
지사목록 = sorted(df['관리지사명'].dropna().unique().tolist())
지사_선택 = Dropdown(options=['전체'] + 지사목록, description='지사선택:')
실행버튼 = Button(description="지도 보기", button_style='info')
output = Output()

# ✅ 5. 실행 함수 정의
def 시각화_지도(b):
    with output:
        output.clear_output()

        필터 = df.copy()
        if 지사_선택.value != '전체':
            필터 = 필터[필터['관리지사명'] == 지사_선택.value]

        if 필터.empty:
            print("❌ 조건에 맞는 고객이 없습니다.")
            return

        m = folium.Map(location=[필터['위도'].mean(), 필터['경도'].mean()], zoom_start=10)

        # 경계
        folium.GeoJson(gdf_sido, name="시도경계").add_to(m)
        folium.GeoJson(gdf_emd, name="읍면동경계").add_to(m)

        # 마커
        cluster = MarkerCluster().add_to(m)
        for _, row in 필터.iterrows():
            color = 'blue' if row['상태'] == '유지' else 'red'
            popup_html = f"""
            <b>상태:</b> {row['상태']}<br>
            <b>지사:</b> {row['관리지사명']}<br>
            <b>계약번호:</b> {row['계약번호']}<br>
            <b>설치주소:</b> {row['설치주소']}<br>
            <b>시군구:</b> {row.get('시군구', '')}<br>
            <b>읍면동:</b> {row.get('읍면동', '')}
            """
            folium.CircleMarker(
                location=[row['위도'], row['경도']],
                radius=5,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(cluster)

        # 저장 및 표시
        m.save("유지해지_고객지도.html")
        display(IFrame("유지해지_고객지도.html", width=950, height=600))

# ✅ 6. 바인딩 및 표시
실행버튼.on_click(시각화_지도)
display(VBox([지사_선택, 실행버튼, output]))

<ipython-input-22-5290d6b86b0a>:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ✅ 유지 vs 해지 고객 지도 시각화 대시보드 (전문가 기법 + Folium + Geo + 필터 UI + 진행률 표시)

import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
from IPython.display import display, IFrame
from ipywidgets import Dropdown, Button, VBox, Output

# ✅ 1. CSV 불러오기
print("[1/6] CSV 파일 불러오기 중...")
df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")
df_u['상태'] = '유지'

df_h = pd.read_csv("/content/drive/MyDrive/지도시각화/해지고객_원본.csv")
df_h['상태'] = '해지'

# ✅ 2. 병합 + 좌표 숫자 변환
print("[2/6] 데이터 병합 및 좌표 정제 중...")
df = pd.concat([df_u, df_h], ignore_index=True)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 3. SHP 파일 불러오기 및 좌표계 변환
print("[3/6] SHP 파일 로딩 중...")
sido_shp_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
gdf_sido = gpd.read_file(sido_shp_path, encoding='cp949')
gdf_sido = gdf_sido.set_crs("EPSG:5179").to_crs("EPSG:4326")

emd_shp_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"
gdf_emd = gpd.read_file(emd_shp_path, encoding='cp949')
gdf_emd = gdf_emd.set_crs("EPSG:5179").to_crs("EPSG:4326")

# ✅ 4. UI 구성
print("[4/6] UI 드롭다운 구성 중...")
지사목록 = sorted(df['관리지사명'].dropna().unique().tolist())
지사_선택 = Dropdown(options=['전체'] + 지사목록, description='지사선택:')
실행버튼 = Button(description="지도 보기", button_style='info')
output = Output()

# ✅ 5. 실행 함수 정의
print("[5/6] 시각화 함수 정의 중...")
def 시각화_지도(b):
    with output:
        output.clear_output()

        필터 = df.copy()
        if 지사_선택.value != '전체':
            필터 = 필터[필터['관리지사명'] == 지사_선택.value]

        print(f"▶️ 선택한 지사: {지사_선택.value}")
        print(f"▶️ 고객 수: {len(필터)}")

        if 필터.empty:
            print("❌ 조건에 맞는 고객이 없습니다.")
            return

        lat_mean = 필터['위도'].mean()
        lon_mean = 필터['경도'].mean()
        print(f"▶️ 평균 좌표: ({lat_mean:.4f}, {lon_mean:.4f})")

        m = folium.Map(location=[lat_mean, lon_mean], zoom_start=11)

        folium.GeoJson(gdf_sido, name="시도경계").add_to(m)
        folium.GeoJson(gdf_emd, name="읍면동경계").add_to(m)

        cluster = MarkerCluster().add_to(m)
        max_points = 500
        print(f"▶️ 최대 {max_points}개 마커만 표시합니다 (전체 {len(필터)} 중)")

        for idx, (_, row) in enumerate(필터.iterrows()):
            if idx >= max_points:
                break

            color = 'blue' if row['상태'] == '유지' else 'red'
            popup_html = f"""
            <b>상태:</b> {row['상태']}<br>
            <b>지사:</b> {row['관리지사명']}<br>
            <b>계약번호:</b> {row['계약번호']}<br>
            <b>설치주소:</b> {row['설치주소']}<br>
            <b>시군구:</b> {row.get('시군구', '')}<br>
            <b>읍면동:</b> {row.get('읍면동', '')}
            """
            folium.CircleMarker(
                location=[row['위도'], row['경도']],
                radius=5,
                color=color,
                fill=True,
                fill_opacity=0.7,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(cluster)

        m.save("유지해지_고객지도.html")
        display(IFrame("유지해지_고객지도.html", width=950, height=600))

# ✅ 6. 바인딩 및 표시
print("[6/6] 인터페이스 렌더링 완료. 실행 준비!")
실행버튼.on_click(시각화_지도)
display(VBox([지사_선택, 실행버튼, output]))


[1/6] CSV 파일 불러오기 중...


<ipython-input-28-83dc428935bc>:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")


[2/6] 데이터 병합 및 좌표 정제 중...
[3/6] SHP 파일 로딩 중...
[4/6] UI 드롭다운 구성 중...
[5/6] 시각화 함수 정의 중...
[6/6] 인터페이스 렌더링 완료. 실행 준비!


In [ ]:
from IPython.display import IFrame
display(IFrame("유지해지_고객지도.html", width=950, height=600))

In [ ]:
import os
for path, dirs, files in os.walk("/content/drive/MyDrive/"):
    for file in files:
        if '유지고객_원본' in file or '해지고객_원본' in file:
            print(os.path.join(path, file))

In [ ]:
df_u.to_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv", index=False)

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point

# ✅ 1. 데이터 불러오기
df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")
df_u['상태'] = '유지'

df_h = pd.read_csv("/content/drive/MyDrive/지도시각화/해지고객_원본.csv")
df_h['상태'] = '해지'

# ✅ 2. 병합 및 정제
df = pd.concat([df_u, df_h], ignore_index=True)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 3. SHP 불러오기
sido_shp_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
gdf_sido = gpd.read_file(sido_shp_path, encoding='cp949').set_crs("EPSG:5179").to_crs("EPSG:4326")

emd_shp_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"
gdf_emd = gpd.read_file(emd_shp_path, encoding='cp949').set_crs("EPSG:5179").to_crs("EPSG:4326")

# ✅ 4. 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=8)

# 행정경계 표시
folium.GeoJson(gdf_sido, name="시도경계").add_to(m)
folium.GeoJson(gdf_emd, name="읍면동경계").add_to(m)

# ✅ 5. 전체 고객 마커 표시 (색상: 유지=파랑, 해지=빨강)
cluster = MarkerCluster().add_to(m)
for _, row in df.iterrows():
    color = 'blue' if row['상태'] == '유지' else 'red'
    popup_html = f"""
    <b>상태:</b> {row['상태']}<br>
    <b>지사:</b> {row['관리지사명']}<br>
    <b>계약번호:</b> {row['계약번호']}<br>
    <b>설치주소:</b> {row['설치주소']}<br>
    <b>시군구:</b> {row.get('시군구', '')}<br>
    <b>읍면동:</b> {row.get('읍면동', '')}
    """
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

# ✅ 6. 저장 및 확인
output_path = "/content/유지해지_전체고객지도.html"
m.save(output_path)
print(f"✅ HTML 저장 완료: {output_path}")

<ipython-input-30-5800137d4b8f>:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")


✅ HTML 저장 완료: /content/유지해지_전체고객지도.html


In [ ]:
from IPython.display import IFrame

IFrame("/content/유지해지_전체고객지도.html", width=1000, height=650)

In [ ]:
import shutil

shutil.copy("/content/유지해지_전체고객지도.html", "/content/drive/MyDrive/지도시각화/유지해지_전체고객지도.html")

'/content/drive/MyDrive/지도시각화/유지해지_전체고객지도.html'

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
import os

# ✅ CSV 불러오기
df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")
df_u['상태'] = '유지'

df_h = pd.read_csv("/content/drive/MyDrive/지도시각화/해지고객_원본.csv")
df_h['상태'] = '해지'

# ✅ 병합 + 좌표 변환
df = pd.concat([df_u, df_h], ignore_index=True)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ SHP 경계파일
sido_path = "/content/drive/MyDrive/행정지도관리/ctprvn_20230729/ctprvn.shp"
emd_path = "/content/drive/MyDrive/행정지도관리/emd_20230729/emd.shp"

gdf_sido = gpd.read_file(sido_path, encoding='cp949').set_crs("EPSG:5179").to_crs("EPSG:4326")
gdf_emd = gpd.read_file(emd_path, encoding='cp949').set_crs("EPSG:5179").to_crs("EPSG:4326")

# ✅ 지사목록
지사목록 = ['중앙지사', '강북지사', '서대문지사', '고양지사', '의정부지사', '남양주지사', '강릉지사', '원주지사']

# ✅ 저장 경로
output_dir = "/content/drive/MyDrive/지도시각화/지사별지도"
os.makedirs(output_dir, exist_ok=True)

# ✅ 지사별 지도 생성
for 지사 in 지사목록:
    df_지사 = df[df['관리지사명'] == 지사].copy()
    if df_지사.empty:
        print(f"❌ {지사} 고객 없음.")
        continue

    m = folium.Map(location=[df_지사['위도'].mean(), df_지사['경도'].mean()], zoom_start=10)
    folium.GeoJson(gdf_sido, name="시도경계").add_to(m)
    folium.GeoJson(gdf_emd, name="읍면동경계").add_to(m)

    cluster = MarkerCluster().add_to(m)
    for _, row in df_지사.iterrows():
        색상 = 'blue' if row['상태'] == '유지' else 'red'
        popup = folium.Popup(f"""
            <b>상태:</b> {row['상태']}<br>
            <b>계약번호:</b> {row['계약번호']}<br>
            <b>설치주소:</b> {row['설치주소']}<br>
            <b>시군구:</b> {row.get('시군구', '')}<br>
            <b>읍면동:</b> {row.get('읍면동', '')}
        """, max_width=300)
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=5,
            color=색상,
            fill=True,
            fill_opacity=0.7,
            popup=popup
        ).add_to(cluster)

    save_path = f"{output_dir}/{지사}_고객지도.html"
    m.save(save_path)
    print(f"✅ 저장 완료: {save_path}")

<ipython-input-36-5c241b59d753>:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv")


✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/중앙지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/강북지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/서대문지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/고양지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/의정부지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/남양주지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/강릉지사_고객지도.html
✅ 저장 완료: /content/drive/MyDrive/지도시각화/지사별지도/원주지사_고객지도.html


In [ ]:
import shutil

shutil.copy("/content/유지해지_전체고객지도.html", "/content/drive/MyDrive/지도시각화/원주지사_고객지도.html")

'/content/drive/MyDrive/지도시각화/원주지사_고객지도.html'

In [ ]:
# 중앙지사만 필터
df_central = df[df['관리지사명'] == '중앙지사'].copy()

# ✅ 최대 500개만 표시 (혹은 1000개)
df_central_sample = df_central.sample(n=500, random_state=42)  # 랜덤 추출

# 지도에 이 데이터만 표시

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
for f in os.listdir("/content/drive/MyDrive/지도시각화"):
    print(f)

유지고객_원본.csv
해지고객_원본.csv
유지해지_전체고객지도.html
지사별지도
중앙지사_고객지도.html
강북지사_고객지도.html
서대문지사_고객지도.html
고양지사_고객지도.html
의정부지사_고객지도.html
남양주지사_고객지도.html
강릉지사_고객지도.html
원주지사_고객지도.html


In [ ]:
!ls "/content/drive/MyDrive/지도시각화"

강릉지사_고객지도.html	  서대문지사_고객지도.html    의정부지사_고객지도.html
강북지사_고객지도.html	  원주지사_고객지도.html      중앙지사_고객지도.html
고양지사_고객지도.html	  유지고객_원본.csv	      지사별지도
남양주지사_고객지도.html  유지해지_전체고객지도.html  해지고객_원본.csv


In [ ]:
import os
os.listdir("/content/drive/MyDrive/지도시각화")

['해지고객_원본.csv',
 '유지고객_원본.csv',
 '지사별지도',
 '유지해지_전체고객지도.html',
 '중앙지사_고객지도.html',
 '강북지사_고객지도.html',
 '서대문지사_고객지도.html',
 '고양지사_고객지도.html',
 '의정부지사_고객지도.html',
 '남양주지사_고객지도.html',
 '강릉지사_고객지도.html',
 '원주지사_고객지도.html']

In [ ]:
import os

# Google Drive 내 /지도시각화 폴더 안 CSV 파일 목록 확인
csv_files = [f for f in os.listdir("/content/drive/MyDrive/지도시각화") if f.endswith(".csv")]
print("✅ 지도시각화 폴더 내 CSV 목록:")
for file in csv_files:
    print("-", file)

✅ 지도시각화 폴더 내 CSV 목록:
- 해지고객_원본.csv
- 유지고객_원본.csv


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# CSV 파일 경로
유지_경로 = "/content/drive/MyDrive/지도시각화/유지고객_원본.csv"
해지_경로 = "/content/drive/MyDrive/지도시각화/해지고객_원본.csv"

# 데이터 불러오기
df_u = pd.read_csv(유지_경로, dtype=str)
df_u["상태"] = "유지"

df_h = pd.read_csv(해지_경로, dtype=str)
df_h["상태"] = "해지"

# 데이터 병합 및 전처리
df = pd.concat([df_u, df_h], ignore_index=True)
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")
df = df.dropna(subset=["위도", "경도"])

# 동대문구 필터링 (최대 500건)
df_dongdaemun = df[df["시군구"].str.contains("동대문", na=False)].head(500)

# 지도 생성
m = folium.Map(location=[df_dongdaemun["위도"].mean(), df_dongdaemun["경도"].mean()], zoom_start=13)
cluster = MarkerCluster().add_to(m)

# 마커 추가
for _, row in df_dongdaemun.iterrows():
    색상 = "blue" if row["상태"] == "유지" else "red"
    popup = f"""<b>상태:</b> {row['상태']}<br>
                <b>지사:</b> {row['관리지사명']}<br>
                <b>계약번호:</b> {row['계약번호']}<br>
                <b>설치주소:</b> {row['설치주소']}"""
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=5,
        color=색상,
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup, max_width=300),
    ).add_to(cluster)

# HTML 파일로 저장
m.save("/content/drive/MyDrive/지도시각화/동대문구_고객지도_500.html")

In [ ]:
df_sample = df_dongdaemun.head(100)

m = folium.Map(location=[df_sample['위도'].mean(), df_sample['경도'].mean()], zoom_start=13)
for _, row in df_sample.iterrows():
    색상 = "blue" if row["상태"] == "유지" else "red"
    popup = f"{row['상태']} | {row['설치주소']} | {row['계약번호']}"
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=4,
        color=색상,
        fill=True,
        fill_opacity=0.7,
        popup=popup
    ).add_to(m)
m.save("/content/drive/MyDrive/지도시각화/동대문구_샘플100.html")

In [ ]:
import folium
from folium.plugins import MarkerCluster

# 예시 데이터프레임: df_dongdaemun
# df_dongdaemun에는 '위도', '경도', '상태', '관리지사명', '계약번호', '설치주소' 등의 컬럼이 포함되어야 합니다.

# 지도 생성
m = folium.Map(location=[df_dongdaemun['위도'].mean(), df_dongdaemun['경도'].mean()], zoom_start=13)

# 마커 클러스터링
cluster = MarkerCluster().add_to(m)

# 마커 추가
for _, row in df_dongdaemun.iterrows():
    색상 = "blue" if row["상태"] == "유지" else "red"
    popup_html = f"""
    <b>상태:</b> {row['상태']}<br>
    <b>지사:</b> {row['관리지사명']}<br>
    <b>계약번호:</b> {row['계약번호']}<br>
    <b>설치주소:</b> {row['설치주소']}
    """
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=5,
        color=색상,
        fill=True,
        fill_color=색상,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

# 지도 저장
m.save("/content/drive/MyDrive/지도시각화/동대문구_고객지도_500.html")

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# CSV 파일 경로
유지_경로 = "/content/drive/MyDrive/지도시각화/유지고객_원본.csv"
해지_경로 = "/content/drive/MyDrive/지도시각화/해지고객_원본.csv"

# 데이터 불러오기
df_u = pd.read_csv(유지_경로, dtype=str)
df_u["상태"] = "유지"

df_h = pd.read_csv(해지_경로, dtype=str)
df_h["상태"] = "해지"

# 데이터 병합 및 전처리
df = pd.concat([df_u, df_h], ignore_index=True)
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")
df = df.dropna(subset=["위도", "경도"])

# 동대문구 필터링 (최대 500건)
df_dongdaemun = df[df["시군구"].str.contains("동대문", na=False)].head(500)

# 지도 생성
m = folium.Map(location=[df_dongdaemun["위도"].mean(), df_dongdaemun["경도"].mean()], zoom_start=13)
cluster = MarkerCluster().add_to(m)

# 마커 추가
for _, row in df_dongdaemun.iterrows():
    색상 = "blue" if row["상태"] == "유지" else "red"
    popup_html = f"""
    <b>상태:</b> {row['상태']}<br>
    <b>지사:</b> {row['관리지사명']}<br>
    <b>계약번호:</b> {row['계약번호']}<br>
    <b>설치주소:</b> {row['설치주소']}
    """
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=5,
        color=색상,
        fill=True,
        fill_color=색상,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

# 지도 저장
m.save("/content/drive/MyDrive/지도시각화/동대문구_고객지도_500.html")

In [ ]:
import pandas as pd
import re

# ✅ CSV 불러오기
df_h = pd.read_csv("/content/drive/MyDrive/지도시각화/해지고객_원본.csv", dtype=str)
df_h['상태'] = '해지'

df_u = pd.read_csv("/content/drive/MyDrive/지도시각화/유지고객_원본.csv", dtype=str)
df_u['상태'] = '유지'

# ✅ 병합
df = pd.concat([df_u, df_h], ignore_index=True)

# ✅ 시군구가 없으면 설치주소에서 자동 추출
def extract_sigungu(address):
    try:
        # 예시 주소: "서울특별시 동대문구 청량리동 11-1"
        match = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|경기|강원|충북|충남|전북|전남|경북|경남|제주)[^\s]*\s([^\s]+구|[^\s]+시|[^\s]+군)', address)
        if match:
            return match.group(2)
    except:
        return None
    return None

# ✅ '시군구' 컬럼 생성
df['시군구'] = df['설치주소'].apply(extract_sigungu)

# ✅ 확인
print(df['시군구'].value_counts().head(10))

# ✅ 저장
df.to_csv("/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv", index=False, encoding='utf-8-sig')
print("✅ 시군구 자동 생성 및 저장 완료: 유지해지_시군구자동추출.csv")

시군구
고양시     8367
남양주시    5682
파주시     4955
원주시     4451
중구      4175
마포구     3979
강릉시     3791
종로구     3693
춘천시     3574
포천시     2997
Name: count, dtype: int64
✅ 시군구 자동 생성 및 저장 완료: 유지해지_시군구자동추출.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

folder_path = "/content/drive/MyDrive/지도시각화"
print(f"✅ '{folder_path}' 폴더 내 파일 목록:")

for file in os.listdir(folder_path):
    print("-", file)

✅ '/content/drive/MyDrive/지도시각화' 폴더 내 파일 목록:
- 해지고객_원본.csv
- 유지고객_원본.csv
- 지사별지도
- 유지해지_전체고객지도.html
- 중앙지사_고객지도.html
- 강북지사_고객지도.html
- 서대문지사_고객지도.html
- 고양지사_고객지도.html
- 의정부지사_고객지도.html
- 남양주지사_고객지도.html
- 강릉지사_고객지도.html
- 원주지사_고객지도.html
- 동대문구_고객지도_500.html
- 동대문구_샘플100.html
- 유지해지_시군구자동추출.csv


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# ✅ CSV 불러오기
path = "/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv"
df = pd.read_csv(path, low_memory=False)  # dtype warning 방지

# ✅ 위도/경도 숫자 변환
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=10)
cluster = MarkerCluster().add_to(m)

# ✅ 마커 표시
for _, row in df.iterrows():
    color = 'blue' if row['상태'] == '유지' else 'red'
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"""
            <b>상태:</b> {row['상태']}<br>
            <b>지사:</b> {row['관리지사명']}<br>
            <b>계약번호:</b> {row.get('계약번호', '')}<br>
            <b>설치주소:</b> {row['설치주소']}<br>
            <b>시군구:</b> {row.get('시군구', '')}<br>
            <b>읍면동:</b> {row.get('읍면동', '')}
        """
    ).add_to(cluster)

# ✅ 지도 저장 및 미리보기
output_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구지도.html"
m.save(output_path)
print(f"✅ 지도 저장 완료: {output_path}")

✅ 지도 저장 완료: /content/유지해지_시군구지도.html


In [ ]:
# ✅ 마커 추가 (색상: 유지=파랑, 해지=빨강, 해지는 크기 더 큼)
for _, row in df.iterrows():
    상태 = row.get('상태', '')
    is_해지 = 상태 == '해지'

    color = 'red' if is_해지 else 'blue'
    radius = 7 if is_해지 else 4  # 해지 고객은 더 크게 표시
    weight = 2 if is_해지 else 1  # 테두리 굵기

    popup_html = f"""
    <b>상태:</b> {'❌ 해지' if is_해지 else '✅ 유지'}<br>
    <b>지사:</b> {row.get('관리지사명', '')}<br>
    <b>계약번호:</b> {row.get('계약번호', '')}<br>
    <b>주소:</b> {row.get('설치주소', '')}<br>
    <b>시군구:</b> {row.get('시군구', '')}<br>
    <b>읍면동:</b> {row.get('읍면동', '')}
    """

    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=radius,
        color=color,
        weight=weight,
        fill=True,
        fill_opacity=0.85,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(cluster)

In [ ]:
import shutil

# 이동 경로 설정
src_path = "/content/마포구_고객지도_유지해지.html"
dst_path = "/content/drive/MyDrive/지도시각화/마포구_고객지도_유지해지.html"

# 파일 이동
shutil.move(src_path, dst_path)
print(f"✅ 지도 파일 이동 완료: {dst_path}")

✅ 지도 파일 이동 완료: /content/drive/MyDrive/지도시각화/마포구_고객지도_유지해지.html


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# ✅ CSV 경로 (구글 드라이브 경로 기준)
csv_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv"
df = pd.read_csv(csv_path)

# ✅ 위경도 정제
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 마포구 필터
df_mapo = df[df['시군구'].str.contains("마포구", na=False)]

# ✅ 지도 생성
m = folium.Map(location=[df_mapo['위도'].mean(), df_mapo['경도'].mean()], zoom_start=13)
cluster = MarkerCluster().add_to(m)

# ✅ 유지/해지 시각화
for _, row in df_mapo.iterrows():
    상태 = row.get('상태', '')
    is_해지 = 상태 == '해지'

    color = 'red' if is_해지 else 'blue'
    radius = 7 if is_해지 else 4
    weight = 2 if is_해지 else 1

    popup = f"""
    <b>상태:</b> {'❌ 해지' if is_해지 else '✅ 유지'}<br>
    <b>지사:</b> {row.get('관리지사명', '')}<br>
    <b>계약번호:</b> {row.get('계약번호', '')}<br>
    <b>주소:</b> {row.get('설치주소', '')}
    """

    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=radius,
        color=color,
        weight=weight,
        fill=True,
        fill_opacity=0.85,
        popup=folium.Popup(popup, max_width=300)
    ).add_to(cluster)

# ✅ 저장 및 미리보기
output_path = "/content/마포구_고객지도_유지해지.html"
m.save(output_path)
print(f"✅ 마포구 지도 저장 완료: {output_path}")

<ipython-input-21-3abd445a5c1a>:7: DtypeWarning: Columns (7,9,18,19,21,23,25,26,28,29,30,31,33,34,35,36,37,44,48,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79,80,81,83,85,86,87,88,89,90,91,92,93,94,96,97,98,108,109,110,111,112,113,116,117,118,119,120,121,122,123,126,127,128,130,131,132,133,134,137,138,139,140,141,143,144,145,146,147,153,155,157,158,162,165,167,168,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ 마포구 지도 저장 완료: /content/마포구_고객지도_유지해지.html


In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from ipywidgets import Dropdown, Button, VBox, Output
from IPython.display import display, IFrame
import os

# ✅ CSV 파일 경로
csv_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv"

# ✅ 저장할 위치 (Google Drive 내 '지도시각화' 폴더)
save_dir = "/content/drive/MyDrive/지도시각화"
save_file = os.path.join(save_dir, "마포구_유지해지_지도.html")

# ✅ 데이터 불러오기
df = pd.read_csv(csv_path)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 마포구 필터링
df_mapo = df[df['시군구'].str.contains("마포", na=False)].copy()

# ✅ 위젯 구성
상태_선택 = Dropdown(
    options=['전체', '유지', '해지'],
    value='전체',
    description='상태 선택:'
)
실행버튼 = Button(description="지도 보기", button_style='info')
output = Output()

# ✅ 실행 함수
def 시각화_마포구(b):
    with output:
        output.clear_output()

        if 상태_선택.value == '전체':
            필터 = df_mapo.copy()
        else:
            필터 = df_mapo[df_mapo['상태'] == 상태_선택.value].copy()

        if 필터.empty:
            print("❌ 조건에 맞는 고객이 없습니다.")
            return

        m = folium.Map(location=[필터['위도'].mean(), 필터['경도'].mean()], zoom_start=13)
        cluster = MarkerCluster().add_to(m)

        for _, row in 필터.iterrows():
            상태 = row['상태']
            color = 'blue' if 상태 == '유지' else 'red'
            radius = 5 if 상태 == '유지' else 9

            popup_html = f"""
                <b>상태:</b> {상태}<br>
                <b>지사:</b> {row.get('관리지사명', '')}<br>
                <b>계약번호:</b> {row.get('계약번호', '')}<br>
                <b>설치주소:</b> {row.get('설치주소', '')}<br>
                <b>시군구:</b> {row.get('시군구', '')}<br>
                <b>읍면동:</b> {row.get('읍면동', '')}<br>
            """
            folium.CircleMarker(
                location=[row['위도'], row['경도']],
                radius=radius,
                color=color,
                fill=True,
                fill_opacity=0.75,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(cluster)

        # ✅ 저장
        m.save(save_file)
        print(f"✅ 저장 완료: {save_file}")
        display(IFrame(save_file, width=950, height=600))

# ✅ UI 바인딩 및 표시
실행버튼.on_click(시각화_마포구)
display(VBox([상태_선택, 실행버튼, output]))

<ipython-input-24-abbe41eea51e>:16: DtypeWarning: Columns (7,9,18,19,21,23,25,26,28,29,30,31,33,34,35,36,37,44,48,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79,80,81,83,85,86,87,88,89,90,91,92,93,94,96,97,98,108,109,110,111,112,113,116,117,118,119,120,121,122,123,126,127,128,130,131,132,133,134,137,138,139,140,141,143,144,145,146,147,153,155,157,158,162,165,167,168,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame
import os

# ✅ CSV 경로 및 저장 경로
csv_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv"
save_path = "/content/drive/MyDrive/지도시각화/마포구_모바일지도.html"

# ✅ 데이터 로드
df = pd.read_csv(csv_path)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 마포구 데이터 필터링
df_mapo = df[df['시군구'].str.contains("마포", na=False)].copy()

# ✅ 지도 생성 (모바일 해상도 고려: 초기 확대, 중심 좌표 보정)
m = folium.Map(location=[df_mapo['위도'].mean(), df_mapo['경도'].mean()], zoom_start=13, control_scale=True)

# ✅ 마커 클러스터 추가
cluster = MarkerCluster().add_to(m)

# ✅ 마커 추가 (모바일 최적화된 팝업 텍스트 + 해지 강조)
for _, row in df_mapo.iterrows():
    상태 = row['상태']
    색상 = 'blue' if 상태 == '유지' else 'red'
    반지름 = 5 if 상태 == '유지' else 9  # 해지는 더 큼

    popup_text = f"{row['상태']} / {row['읍면동'] or ''}"
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=반지름,
        color=색상,
        fill=True,
        fill_opacity=0.8,
        popup=popup_text
    ).add_to(cluster)

# ✅ 지도 저장
m.save(save_path)
print(f"✅ 모바일 지도 저장 완료: {save_path}")
display(IFrame(save_path, width="100%", height=500))

<ipython-input-25-76c3e1ba8fb2>:12: DtypeWarning: Columns (7,9,18,19,21,23,25,26,28,29,30,31,33,34,35,36,37,44,48,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79,80,81,83,85,86,87,88,89,90,91,92,93,94,96,97,98,108,109,110,111,112,113,116,117,118,119,120,121,122,123,126,127,128,130,131,132,133,134,137,138,139,140,141,143,144,145,146,147,153,155,157,158,162,165,167,168,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ 모바일 지도 저장 완료: /content/drive/MyDrive/지도시각화/마포구_모바일지도.html


In [ ]:
from google.colab import drive
import os

# ✅ Google Drive 마운트
drive.mount('/content/drive')

# ✅ 지도시각화 폴더 경로
folder_path = "/content/drive/MyDrive/지도시각화"

# ✅ 폴더 내 파일 목록 출력
print("✅ '/content/drive/MyDrive/지도시각화' 폴더 내 파일 목록:")
for file in os.listdir(folder_path):
    print("-", file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ '/content/drive/MyDrive/지도시각화' 폴더 내 파일 목록:
- 해지고객_원본.csv
- 유지고객_원본.csv
- 지사별지도
- 유지해지_전체고객지도.html
- 중앙지사_고객지도.html
- 강북지사_고객지도.html
- 서대문지사_고객지도.html
- 고양지사_고객지도.html
- 의정부지사_고객지도.html
- 남양주지사_고객지도.html
- 강릉지사_고객지도.html
- 원주지사_고객지도.html
- 동대문구_샘플100.html
- 동대문구_고객지도_500.html
- 유지해지_시군구자동추출.csv
- 유지해지_시군구지도.html
- 마포구_고객지도_유지해지.html
- 마포구_유지해지_지도.html
- 마포구_모바일지도.html
- 의원_정리본.csv


In [ ]:
import pandas as pd

# 예시: 파일명을 확인 후 여기에 정확히 입력
file_path = "/content/drive/MyDrive/지도시각화/의원_정리본.csv"
df = pd.read_csv(file_path)

# 파일 구조 미리보기
df.head()

,사업장명,소재지전체주소,영업상태명,시군구,읍면동,위도,경도
0,바른홍치과의원,경기도 부천시 원미구 중동 678 3층 일부호,영업/정상,부천시,중동,37.496974,126.761923
1,은하여성의원,서울특별시 강동구 성내동 415-7 동성빌딩,영업/정상,서울특별시,성내동,37.529685,127.136571
2,공릉온치과의원,서울특별시 노원구 공릉동 576-25 동흥빌딩,영업/정상,서울특별시,공릉동,37.621964,127.073839
3,나무와흙한의원,충청남도 공주시 반포면 학봉리 289번지 3호,영업/정상,공주시,반포면,36.361804,127.252502
4,시카고웰빙치과의원,NaN,영업/정상,NaN,NaN,37.595909,127.085999


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# ✅ CSV 불러오기
path = "/content/drive/MyDrive/지도시각화/의원_정리본.csv"
df = pd.read_csv(path)

# ✅ 위도/경도 숫자 변환
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=11)
cluster = MarkerCluster().add_to(m)

# ✅ 마커 추가
for _, row in df.iterrows():
    popup_text = f"""
    <b>의원명:</b> {row.get('사업장명', '')}<br>
    <b>주소:</b> {row.get('소재지전체주소', '')}<br>
    <b>상태:</b> {row.get('영업상태명', '')}
    """
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color='green' if row.get('영업상태명') == '영업/정상' else 'gray',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(cluster)

# ✅ 저장 및 확인
output_path = "/content/drive/MyDrive/지도시각화/의원_지도.html"
m.save(output_path)
print(f"✅ 지도 저장 완료: {output_path}")

✅ 지도 저장 완료: /content/drive/MyDrive/지도시각화/의원_지도.html


In [ ]:
# 이미 완료된 부분이라면 생략 가능
output_path = "/content/drive/MyDrive/지도시각화/의원_지도.html"
m.save(output_path)

In [ ]:
import qrcode
qrcode.make("https://drive.google.com/uc?export=view&id=1abcXYZ12345678").save("의원_지도_QR.png")

ModuleNotFoundError: No module named 'qrcode'

In [7]:
# ✅ 1. 필수 라이브러리 설치 및 임포트
!pip install geopandas folium gdown --quiet

import pandas as pd
import folium
from folium.plugins import MarkerCluster
from google.colab import drive

# ✅ 2. Google Drive 마운트
drive.mount('/content/drive')

# ✅ 3. CSV 파일 경로 설정
file_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구자동추출.csv"

# ✅ 4. 데이터 불러오기 및 전처리
df = pd.read_csv(file_path)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df = df.dropna(subset=['위도', '경도'])

# ✅ 5. Folium 지도 생성
m = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], zoom_start=11)
cluster = MarkerCluster().add_to(m)

for _, row in df.iterrows():
    상태 = row.get('상태', '유지')
    color = 'blue' if 상태 == '유지' else 'red'
    radius = 6 if 상태 == '해지' else 4

    popup = folium.Popup(f"""
    <b>상태:</b> {상태}<br>
    <b>지사:</b> {row.get('관리지사명', '')}<br>
    <b>주소:</b> {row.get('설치주소', '')}<br>
    <b>시군구:</b> {row.get('시군구', '')}<br>
    <b>읍면동:</b> {row.get('읍면동', '')}
    """, max_width=300)

    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=radius,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=popup
    ).add_to(cluster)

# ✅ 6. 결과 저장
output_path = "/content/drive/MyDrive/지도시각화/유지해지_시군구지도_모바일.html"
m.save(output_path)
print(f"✅ 지도 저장 완료: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-7-e44e6f9103c3>:16: DtypeWarning: Columns (7,9,18,19,21,23,25,26,28,29,30,31,33,34,35,36,37,44,48,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,74,75,76,77,78,79,80,81,83,85,86,87,88,89,90,91,92,93,94,96,97,98,108,109,110,111,112,113,116,117,118,119,120,121,122,123,126,127,128,130,131,132,133,134,137,138,139,140,141,143,144,145,146,147,153,155,157,158,162,165,167,168,170,171,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


✅ 지도 저장 완료: /content/drive/MyDrive/지도시각화/유지해지_시군구지도_모바일.html
